# SQLAlchemy Example
An example of object-relational mapping (ORM) using SQLAlchemy.

#### Import packages and modules used in subsequent cells.

In [2]:
import configparser
import sqlalchemy
import sqlalchemy_utils

#### Define a function for parsing database connection parameters from a configuration file.

In [3]:
def _database_config(config_file, section='postgresql'):

    # create a parser
    config_parser = configparser.ConfigParser()

    # read configuration file
    config_parser.read(config_file)

    # get section, default to postgresql
    db_config = {}
    if config_parser.has_section(section):
        params = config_parser.items(section)
        for param in params:
            db_config[param[0]] = param[1]
    else:
        raise Exception(f'Section {section} not found in the {config_file} file')

    return db_config

#### Read database connection parameters and use these to configure an SQLAlchemy Engine instance.

In [4]:
params = _database_config("C:/home/data/pullpoint/database.ini")
db_connection_details = f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['database']}"
engine = sqlalchemy.create_engine(db_connection_details, echo=True)

#### Create an instance of the declarative base class and define an associated mapped class for notifications.

In [5]:
Base = sqlalchemy.ext.declarative.declarative_base()

class Notification(Base):
    
    __tablename__ = 'notifications'
    
    # object attributes (columns in each row)
    notification_id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True, autoincrement=True)
    ip_address = sqlalchemy.Column(sqlalchemy_utils.IPAddressType)
    message = sqlalchemy.Column(sqlalchemy.Unicode(255))
    
    # simple representation
    def __repr__(self):
        return f"<Notification(notification_id={self.notification_id}, ip_address={self.ip_address}, message={self.message})>"

#### MetaData and Table objects
The Notifications class we've declared defines metadata information about a corresponding table in our database. We now have a `Table` object (for our `notifications` table) which is part of a `MetaData` registry. The MetaData object is available from our declarative base object as an attribute, `.metadata`, and it includes the ability to emit a limited set of schema generation commands to the database. The `notifications` table can now be created by calling the MetaData.create_all() method, passing in our `Engine` instance as a source of database connectivity.

In [6]:
Base.metadata.create_all(engine)

2019-02-11 16:49:52,595 INFO sqlalchemy.engine.base.Engine select version()
2019-02-11 16:49:52,597 INFO sqlalchemy.engine.base.Engine {}
2019-02-11 16:49:52,683 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-02-11 16:49:52,684 INFO sqlalchemy.engine.base.Engine {}
2019-02-11 16:49:52,770 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-02-11 16:49:52,771 INFO sqlalchemy.engine.base.Engine {}
2019-02-11 16:49:52,833 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-02-11 16:49:52,833 INFO sqlalchemy.engine.base.Engine {}
2019-02-11 16:49:52,927 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-02-11 16:49:52,928 INFO sqlalchemy.engine.base.Engine {}
2019-02-11 16:49:53,017 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

#### Create an instance of the mapped class and insert a corresponding record into the corresponding database table
We will create a `Session` class that will be bound to the database via the `Engine` instance, and we'll use an instance of this `Session` class to access the database via connections from a pool maintained by the associated `Engine`.

In [7]:
initial_notification = Notification(ip_address="12.34.56.78", message="First notification message.")
second_notification = Notification(ip_address="12.34.56.79", message="Second notification message.")

Session = sqlalchemy.orm.sessionmaker(bind=engine)
session = Session()

session.add(initial_notification)
session.add(second_notification)

At this point the notification object hasn't been saved as a row in the database table, since the instance is **pending**, i.e. the SQL to persist the notification object won't be issued until it is needed, using a process known as a **flush**. For example, if we perform a lookup of the notification via a query then the `Session` will first flush all pending information before issuing the query SQL.

Below we'll query for all `Notification` objects (i.e. rows in the `notifications` table) filtered by the IP address attribute we've used for the initial notification, "12.34.56.78", and get the first one found in the list (and in this case there should be only one). This will trigger the flushing of the pending insertions from the above `Session.add()` calls, which will happen before the SQL for the query is issued.

In [8]:
notification = session.query(Notification).filter_by(ip_address="12.34.56.78").first()
notification

2019-02-11 17:22:07,487 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-11 17:22:07,488 INFO sqlalchemy.engine.base.Engine INSERT INTO notifications (ip_address, message) VALUES (%(ip_address)s, %(message)s) RETURNING notifications.notification_id
2019-02-11 17:22:07,488 INFO sqlalchemy.engine.base.Engine {'ip_address': '12.34.56.78', 'message': 'This is a notification message.'}
2019-02-11 17:22:07,652 INFO sqlalchemy.engine.base.Engine SELECT notifications.notification_id AS notifications_notification_id, notifications.ip_address AS notifications_ip_address, notifications.message AS notifications_message 
FROM notifications 
WHERE notifications.ip_address = %(ip_address_1)s 
 LIMIT %(param_1)s
2019-02-11 17:22:07,653 INFO sqlalchemy.engine.base.Engine {'ip_address_1': '12.34.56.78', 'param_1': 1}


<Notification(notification_id=1, ip_address=12.34.56.78, message=This is a notification message.)>